In [12]:
import pandas as pd
import pickle
import os

In [13]:
from utils import haversine
from utils import join_data, train_test,hervy_dist, minutes_convert, haversine,taxi_demand_cluster, predict_lat_long, plt_map

In [14]:
dst_folder = os.path.join(os.getcwd(),"Processed_Data")

In [15]:
df = join_data(dst_folder)

In [16]:
df_model = pd.read_csv("Classical_ML_Models_test.csv")

In [17]:
df_model.drop("Unnamed: 0", axis=1, inplace=True)

In [18]:
df_model.sort_values(by= "Score", inplace=True)

In [19]:
df_model

,Model,Score,File_name
4,XGB,1890.300460,finalized_model_XGB.sav
3,Random_Forest,1897.904023,finalized_model_random_forest.sav
0,Linear_Regression,2240.509841,finalized_linear.sav
2,Decision_tree,2336.103194,finalized_model_decision_tree.sav
5,XGBF,2355.207638,finalized_model_random_forest.sav
1,KNN,2630.195285,finalized_model_knn.sav


In [20]:
filename = df_model.iloc[0]["File_name"]

In [43]:
df_test  = pd.read_csv("Test_Data.csv")
y_test = pd.read_csv("Y_Test.csv")
df_test= df_test.reset_index()
y_test = y_test.reset_index()


In [44]:
df_test= df_test.drop(["index","Unnamed: 0"], axis=1)
y_test= y_test.drop(["index","Unnamed: 0"], axis=1)

In [12]:
df_test = df_test[:100]
y_test = y_test[:100]

In [13]:
test_data = df_test.copy()

In [30]:
model = pickle.load(open(filename, 'rb'))
scaler = pickle.load(open('scaler.pkl','rb'))

In [17]:
def closest_prediction(df,df_test,model,index, scaler):
#     df_test.drop("Unnamed: 0",axis=1, inplace=True)
    test_data = df_test.copy()
    df_test["Latitude"] = df_test["Latitude"]/90
    df_test["Longitude"] = df_test["Longitude"]/180
    df_test[['Hour', 'Day', 'Minutes_Identifier']] = scaler.fit_transform(df_test[['Hour', 'Day', 'Minutes_Identifier']])
    test_lat = df_test.iloc[[index]]
    pred = model.predict(test_lat)
#     print(pred)
    distance_to_prediction = haversine(list(pred[0]),list(test_data[["Latitude","Longitude"]].iloc[[index]].values[0]))
    df_cluster, df_centroids = taxi_demand_cluster(df,test_data.iloc[index]["Hour"], 0,test_data.iloc[index]["Day"], 300, 20)
    dfc, df_label = predict_lat_long(list(test_data[["Latitude","Longitude"]].iloc[[0]].values[0]), df_centroids)
    if dfc.empty:
        prediciton = (pred[0][0],pred[0][1])
    else:
        if dfc.iloc[0]["Distance"] <=distance_to_prediction:
            prediciton = (dfc["Latitude_1"].iloc[0],dfc["Longitude_1"].iloc[0])
        else:
            prediciton = (pred[0][0],pred[0][1])
    return(prediciton)

In [53]:
predicitons=[]
for each in range(df_test.shape[0]):
    predicitons.append(closest_prediction(df.copy(),df_test.copy(), model, each, scaler))
    print(each)

In [54]:
df_pred = pd.DataFrame(predicitons)

In [31]:
df_pred.columns =["Latitude_pre", "Longitude_pre"]

In [26]:
df_final = pd.concat([df_test, y_test], axis=1)